In [6]:
import cv2
from sklearn.cluster import KMeans
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC  # Example weak learner
from sklearn.model_selection import train_test_split
import os
import numpy as np
from cv2.xfeatures2d import SIFT_create as sift_create

# Define paths and hyperparameters
data_path = "train-dataset"
k = 100  # Number of visual words (adjust as needed)

# Load image paths and labels
images = []
labels = []
for folder_name in os.listdir(data_path):
  folder_path = os.path.join(data_path, folder_name)
  for filename in os.listdir(folder_path):
    image_path = os.path.join(folder_path, filename)
    label = folder_name  # Assuming folder name represents class label
    images.append(image_path)
    labels.append(label)

# Function to extract SURF features
def extract_surf_features(image_path):
  img = cv2.imread(image_path)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  surf = sift_create()
  keypoints, descriptors = surf.detectAndCompute(gray, None)
  return descriptors

# Extract features from training images
all_features = []
for image_path in images:
  features = extract_surf_features(image_path)
  all_features.append(features)


# Create the visual vocabulary using k-means
kmeans = KMeans(n_clusters=k)
kmeans.fit(np.concatenate(all_features))  # Combine all descriptors

KMeans(n_clusters=100)

In [8]:
# Function to create image histograms
kmeans.cluster_centers_ = kmeans.cluster_centers_.astype(np.double)
    
def create_image_histogram(image_path, kmeans):
  descriptors = extract_surf_features(image_path)
  histogram = np.zeros(k)
  for descriptor in descriptors:
    cluster_id = kmeans.predict([descriptor])[0]
    histogram[cluster_id] += 1
  return histogram

# Create histograms for training images
image_features = []
for image_path in images:
  histogram = create_image_histogram(image_path, kmeans)
  image_features.append(histogram)

ValueError: Invalid parameter 'base_estimator' for estimator AdaBoostClassifier(random_state=42). Valid parameters are: ['algorithm', 'estimator', 'learning_rate', 'n_estimators', 'random_state'].

In [10]:
# Train the SVM classifier
clf = AdaBoostClassifier(n_estimators=50, random_state=42)
clf.set_params(algorithm="SAMME")
clf.set_params(estimator=SVC(kernel='linear'))  # Set the base learner
clf.fit(np.array(image_features), labels)

AdaBoostClassifier(algorithm='SAMME', estimator=SVC(kernel='linear'),
                   random_state=42)

In [16]:
# Function to classify a new image
def classify_image(image_path, kmeans, clf):
  histogram = create_image_histogram(image_path, kmeans)
  prediction = clf.predict([histogram])[0]
  return prediction

wrong = 0
for i in range(1 , 1000):
  new_image_path = f"fonts-dataset/IBM Plex Sans Arabic/{i}.jpeg"
  predicted_class = classify_image(new_image_path, kmeans, clf)
  print("Predicted class for", new_image_path, ":", predicted_class)
  if (predicted_class != "IBM Plex Sans Arabic"):
    wrong = wrong + 1

Predicted class for fonts-dataset/IBM Plex Sans Arabic/1.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/2.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/3.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/4.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/5.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/6.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/7.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/8.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/9.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/10.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/11.jpeg : IBM Plex Sans Arabic
Predicted class for fonts-dataset/IBM Plex Sans Arabic/12.jpeg 